In [13]:
#!pip install openai
#!pip install langchain
#!pip install docarray

openai                                   1.3.8


# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)

You can predict outputs from both LLMs and ChatModels:

In [ ]:
chat_model.predict("hi!")
# Output: "Hi"

You can also use the predict method over a string input:

In [ ]:
text = "What would be a good name for a dog that loves to nap??"


chat_model.predict(text)
# Output: "Snuggles"

Finally, you can use the `predict_messages` method over a list of messages:

In [ ]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text)]

chat_model.predict_messages(messages)

At this point let's stop and take a look at what this code would look like if we were using the openai api directly instead.

Let's understand what is going on.

Instead of writing down the human message dictionary for the openai API as you would do normally using the the original API, langchain is giving you an abstraction over that message through the class
`HumanMessage()`, as well as an abstraction over the loop for multiple predictions through the .`predict_messages()` method.

Now, why is that an useful thing?

Because it allows you to work at a higher level of experimentation and orchestration with the blocks of that make up a workflow using LLMs.

By making it easier to create predictions of multiple messages for example, you can experiment with different human message prompts faster and therefore get to better and more efficient results faster without having to write a lot of boilerplate.

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

# Chains

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.run("sleeping")

You can also create more complex ChatPromptTemplates that contains a list of ChatMessageTemplates:

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [ ]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

**LLMChain**

Finally, you can combine all these components into an LLMChain:

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generated 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("dogs")
# Output: ['Golden Retriever','Labrador Retriever','German Shepherd','Bulldog','Poodle']

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

One of the best applications of langchain is for the "chat with your data"-types of applications, where the user uploads a document like a pdf or a .txt file, and is able to query that document using langchain powered by an LLM like ChatGPT. 

# LangChain Lab Exercises

Let's take a look at a simple example of **Simple Sequential Chain**:

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

In [ ]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = ChatOpenAI(temperature=.7)
template = """You are a learning assistant. Given a technical subject, write down 5 fundamental concepts to understand it.
Subject: {subject}
Learning assistant: The 5 fundamental concepts are:"""
prompt_template = PromptTemplate(input_variables=["subject"], template=template)
learning_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = ChatOpenAI(temperature=.7)
template = """You are an expert teacher in all technical and scientific fields. Given a list of 5 concepts, write down a simple intuitive explanation of each concept.
Concepts:
{concepts}
Intuitive explanations:"""
prompt_template = PromptTemplate(input_variables=["concepts"], template=template)
explanation_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is the overall chain where we run these two chains in sequence.
learning_overall_chain = SimpleSequentialChain(chains=[learning_chain, explanation_chain], verbose=True)

In [ ]:
output = learning_overall_chain.run("3D printing")
output

**Sequential Chains**

In [ ]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = ChatOpenAI(temperature=.7)
template = """You are a learning assistant. Given a technical subject, write down 5 fundamental concepts to understand it.
Subject: {subject}
Field: {field}
Learning assistant: The 5 fundamental concepts are:"""
prompt_template = PromptTemplate(input_variables=["subject", "field"], template=template)
learning_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="concepts")

In [ ]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = ChatOpenAI(temperature=.7)
template = """You are an expert teacher in all technical and scientific fields. Given a list of 5 concepts, write down a simple intuitive explanation of each concept.

Concepts:
{concepts}
Intuitive explanations:"""
prompt_template = PromptTemplate(input_variables=["concepts"], template=template)
explanation_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="explanations")

In [ ]:
# This is the overall chain where we run these two chains in sequence.
learning_overall_chain = SequentialChain(
    chains=[learning_chain, explanation_chain],
    input_variables=["subject", "field"],
    # Here we return multiple variables
    output_variables=["concepts", "explanations"],
    verbose=True)

In [ ]:
learning_overall_chain({"subject":"3D Printing", "field": "engineering"})

# Simple Q&A Example

In [ ]:
#!pip install docarray
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
import pandas as pd
df = pd.read_csv("./superheroes.csv")
df.head()

In [ ]:
file = 'superheroes.csv'
loader = CSVLoader(file_path=file)

Now, let's set up our Vector store (we'll talk about what that is in a second):

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

In [ ]:
query = "Tell me the catch phrase for Captain Thunder"

In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

Ok, cool! So, now let's backtrack a little bit and discuss what is going on.

If we want LLMs to get access to our data in order to help us get insights, we have one major problem: LLMs have a limited context window, meaning they can only process a few thousand words at a time which constraints their ability to answer questions (for example) of really big documents. 

That's where things like embeddings and vector stores come into play, these are way of representing the data in such a way to facilitate the access by an LLM. Let's break it down, starting with embeddings:

Embeddings are numerical representations of data, meaning, their a way to represent data such that the semantic information of that data is properly reflected in the distances between the data points in the embedding.

That means that text with similar content will have similar vectors (which is a way of saying that they'll be closer together in the embedding).

![](2023-07-30-19-29-27.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

Now, let's talk about vector databases.

A vector database is a way to store these embeddings, these numerical representations that we just discussed.

The pipeline is:
- In coming document
- Create chunks of text from that document
- Embed each chunk
- Store these embeddings

![](2023-07-30-19-32-13.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

Now, we can query those embeddings stored in the vector database to get the most relevant responses!

So when we create a query, that query is first embedded and then we compare its embedding with the embeddings we have stored in the vector database. We then select the N-most similar embeddings, and pass those to the LLM.

![](images/2023-07-30-19-34-48.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

- __Embedding__: the useful data representation that will be used as the thing (or things) we can query
- __Vector Database__: the vectorization of the embedding chunks (the database for the embeddings where we can do the query)

### Embeddings


![](./images/embeddings.png)

![](./images/embeddings2.png)


### Vector Database




![](./images/2023-07-17-12-48-28.png)



# References
- https://python.langchain.com/docs/get_started/introduction.html
- https://medium.com/@remitoffoli/a-visual-guide-to-llm-powered-app-architecture-57e47426a92f
- [LangChain for LLM App Development short course by coursera](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [LLM Evaluation](https://learn.deeplearning.ai/langchain/lesson/6/evaluation)
[Models, Prompts, parsers, memory and chains from this langchain for](https://learn.deeplearning.ai/langchain/lesson/7/agents)
- [Chat With Your Data - Retrieval](https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/5/retrieval)
- [Emebeddings simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [Vector DBs - simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)